# 7. 가설과 추론

Goal: 지금까지 배웠던 통계와 확률 이론으로 가설을 검정하고 통계적으로 추론을 해 보자

7.1 통계적 가설검정

가설(hypothesis)의 예: 동전은 앞뒤가 나올 확률이 공평하다 / 데이터과학자는 알보다 파이썬을 선호한다 등의 주장

H0 vs. H1
기본적인 가설을 의미하는 귀무가설 (H0, Null Hypothesis)와 비교하고 싶은 대립가설 (H1, alternative hypothesis)
통계를 사용해서 H0을 기각할지 말지를 결정

In [2]:
from probability import normal_cdf, inverse_normal_cdf # 이전장에서 작업한 확률 코드를 임포트 해야함
import math, random

7.2 동전 던지기 예시

동전이 하나 있는데, 공평한 동전인지 아닌지 검정하고 싶음.

H0: p = 0.5 vs H1: p != 0.5

가정: 확률변수가 정규분포를 따른다는 가정

In [5]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [7]:
# 누적분포함수는 확률변수가 특정 값보다 작은 확률 나타냄
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
# 만약 확률변수가 특정 값보다 작지 않다면, 특정 값보다 크다는 것을 의미
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)
    
# it's between if it's less than hi, but not less than lo
# 만약 확률변수가 hi보다 작고 lo보다 작지 않다면, 확률변수는 hi와 low사이에 존재
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
# 만약 확률 변수가 범위 밖에 존재한다면, 범위 안에 존재하지 않는것을 의미
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

"""반대로 확률이 주어졌을 때, 평균을 중심으로 하는 (대칭) 구간을 구할 수 있음.
예를 들어, 분포의 60%를 차지하는 평균 중심의 구간을 구하고 싶으면, 양쪽 꼬리 부분이 각각 분포의 20%차지하는 부분을 구하면 됨"""

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)
    
def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds 
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [11]:
#실제로 동전을 1000번 던져보기. 동전이 공평하면 X = 500 표준편차는 15.8

mu_0, sigma_0 = normal_approximation_to_binomial(1000,0.5)

mu_0, sigma_0

(500.0, 15.811388300841896)

1종 오류를 얼마나 허용해 줄 것인지를 의미하는 유의수준(sig) 결정

![Image of Errors](http://data.newdaily.co.kr/data/photos/20150939/shp_1443320553.jpg)

In [13]:
#Significant를 0.5로 설정, p가 참이면 X가 주어진 범위에서 벗어날 확률은 5%이하 = 20번중에 19은 맞는 결과

normal_two_sided_bounds(0.95, mu_0, sigma_0) 

(469.01026640487555, 530.9897335951244)

In [18]:
#2종오류를 범하지 않을 확률을 구하려면 검정력 (power)를 알 수 있음
#2종오류는 H0이 거짓이지만 기각하지 않을때

#p가 0.5라고 가정 할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

#p가 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

#제2종 오류는, 귀무가설 (h0)를 기각하지 못한다는 의미, 즉 X가 구간 안에 존재
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

power = 1 - type_2_probability

print(power)

0.886548001295


한편, p <= 0.5, 즉 동전이 앞면에 편향되지 않는 경우를 귀무가설로 정하면 X가 50보다 크면 귀무가설을 기각하고 50보다 작으면 기각하지 않는 단측 검증 필요

유의수준이 5%인 가설검정을 위해서는 normal_probability_below를 사용하여 분포의 95%가 해당값 이하는 경계 값을 찾을 수 있다.

In [23]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print(hi) #( < 531, 분포 상위 부분에 더 높은 확률을 주기 위해)

526.007358524


In [26]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

0.936379480331


이 가설검정은 X가 469보다 작을 때 H0를 기각하는게 아니라(H1이 참이면
이는 거의 발생 안함) X가 526~531 사이 일 때,
H0를 기각하기 때문에 (H1이 참이라면 이는 가끔 발생)
전보다 검정력이 더 좋아 졌다고 볼 수 있다.

## 7.3 P-value
어떤 확률 값을 기준으로 구간을 선택하는 대신, H0가 참이라고 가정하고 실제로 관측된 값보다 더 극단적인 값이 나올 확률을 구하는 것임.

동전이 공평한지 판단하기 위해 양측 검증을 하자

In [27]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is above x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is below x
        return 2 * normal_probability_below(x, mu, sigma)   

In [28]:
#만약 동전의 앞면의 530번 관측되면, p-value는 아래와 같음
# 530보다 529.5를 사용한 것은 연속수정 (continuity correction* ?)일 때임


two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598857

In [32]:
def count_extreme_values():
    extreme_value_count = 0
    for _ in range(100000):
        num_heads = sum(1 if random.random() < 0.5 else 0    # count # of heads
                        for _ in range(1000))                # in 1000 flips
        if num_heads >= 530 or num_heads <= 470:             # and count how often
            extreme_value_count += 1                         # the # is 'extreme'

    return extreme_value_count / 100000

In [34]:
count_extreme_values() #0.062
# p값이 0.05보다 크게 나왔기 때문에 H0를 기각하지 않는다. 만약에 동전의 앞면이 532번
# 나왔으면 0.05보다 작고, 이 경우는 귀무 가설을 기각

0

7.4 신뢰구간

분포를 모른다면, Confidence Interval을 사용하여 가설 검증 가능.


In [43]:
p = 0.525

math.sqrt(p * (1-p) / 1000)

0.015791611697353755

In [35]:
#p의 정확한 값을 모르므로 추정치를 사용 가능

p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

False

In [37]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

False

In [40]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

TypeError: normal_probability_above() takes at least 1 argument (0 given)

In [41]:
def run_experiment():
    """flip a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [14]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]

[sum(experiment) for experiment in experiments][:10]

[508, 491, 484, 483, 502, 499, 483, 511, 481, 538]

In [15]:
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])
num_rejections

46

In [16]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [17]:
def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # no weight outside of [0, 1]    
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

In [18]:
z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [19]:
two_sided_p_value(z) # 두 광고 효과의 평균이 같을 확률이 높음 (p > 0.05) = 광고효과가 같음

0.254141976542236

In [20]:
z = a_b_test_statistic(1000, 200, 1000, 150)
z

-2.948839123097944

In [21]:
two_sided_p_value(z) # 두 광고 효과의 평균이 같을 확률이 매우 낮음 (p < 0.05) = 광고효과가 다름

0.003189699706216853

In [22]:
##
#
# Bayesian Inference
#
##

def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # no weight outside of [0, 1]    
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)